


![](docs/attachments/solarMED_binary_tree.png)

In [1]:
import pandas as pd
import numpy as np
from loguru import logger
import json
from pathlib import Path

%load_ext autoreload
%autoreload 2

from solarMED_optimization.path_exploration import (
    Node, 
    generate_edges,
    generate_edges_dataframe
)

from solarMED_optimization import convert_to_state
from solarMED_modeling import SF_TS_State, MedState, SolarMED_State

Np = 5 # Prediction horizon

In [2]:
# Read json file with path from individual systems

with open(Path('results') /f'all_paths_SFTS_Np_{Np}.json', 'r') as f:
    all_paths = json.load(f)
    
# Convert to the correct type
all_paths_sfts = [[convert_to_state(state, state_cls=SF_TS_State) for state in path] for path in all_paths]

# Read json file
with open(Path('results') /f'all_paths_MED_Np_{Np}.json', 'r') as f:
    all_paths = json.load(f)

# Convert to the correct type
all_paths_med = [[convert_to_state(state, state_cls=MedState) for state in path] for path in all_paths] # Terrible

In [3]:
# Case 1. Given that both systems are evaluated with the same sample rate, for every step of one, all the states from the other system are possible
# Case 2. If they had different sample rates, the states list would have as many elements as samples in the faster system. For every fast sample the states of the slow one would remain constant, and for every slow sample, every state of the slow one would be possible 

from recursitron import dump_as

# Case 1.

all_paths_str = []
all_paths = []
for idx, sf_ts_path in enumerate(all_paths_sfts):
    for med_path in all_paths_med:
        path_str = [f'{sfts_state.value}{med_state.value}' for sfts_state, med_state in zip(sf_ts_path, med_path)]
        
        path = [SolarMED_State(state_str) for state_str in path_str]
        # path = [convert_to_state(state, state_cls=SolarMED_State) for state in states_str]
        
        all_paths_str.append(path_str)
        all_paths.append(path)
        
    logger.info(f'Path {idx}/{len(all_paths_sfts)} ({idx/len(all_paths_sfts)*100:.0f}%) done')
    
dump_as(all_paths, file_path=Path('results')/f'all_paths_SolarMED_Np_{Np}', file_format='json')

2024-05-02 15:39:16.386 | INFO     | __main__:<module>:20 - Path 0/179 (0%) done
2024-05-02 15:39:16.387 | INFO     | __main__:<module>:20 - Path 1/179 (1%) done
2024-05-02 15:39:16.388 | INFO     | __main__:<module>:20 - Path 2/179 (1%) done
2024-05-02 15:39:16.389 | INFO     | __main__:<module>:20 - Path 3/179 (2%) done
2024-05-02 15:39:16.391 | INFO     | __main__:<module>:20 - Path 4/179 (2%) done
2024-05-02 15:39:16.392 | INFO     | __main__:<module>:20 - Path 5/179 (3%) done
2024-05-02 15:39:16.393 | INFO     | __main__:<module>:20 - Path 6/179 (3%) done
2024-05-02 15:39:16.394 | INFO     | __main__:<module>:20 - Path 7/179 (4%) done
2024-05-02 15:39:16.395 | INFO     | __main__:<module>:20 - Path 8/179 (4%) done
2024-05-02 15:39:16.397 | INFO     | __main__:<module>:20 - Path 9/179 (5%) done
2024-05-02 15:39:16.398 | INFO     | __main__:<module>:20 - Path 10/179 (6%) done
2024-05-02 15:39:16.399 | INFO     | __main__:<module>:20 - Path 11/179 (6%) done
2024-05-02 15:39:16.401 | 

In [4]:
Node(step_idx=0, state=SolarMED_State('000'))

Node(step_idx=0, state=<SolarMED_State.sf_IDLE_ts_IDLE_med_OFF: '000'>, node_id='step000_000', state_value='000', state_name='sf_IDLE_ts_IDLE_med_OFF', x_pos=0, y_pos=0)

In [5]:
# Build states/nodes/vertices dataframe

state_cls = SolarMED_State

base_list = [str(state.value) for state in state_cls]
N_nodes = Np*len(base_list)

result = [
    Node(
        step_idx=step_idx,
        state=state,
    ).model_dump()
    for step_idx in range(Np) for state in [state for state in state_cls]
]


nodes_sfts_df = pd.DataFrame([
    Node(step_idx=step_idx,state=state).model_dump()
    for step_idx in range(Np) for state in [state for state in SF_TS_State]
])
nodes_med_df = pd.DataFrame([
    Node(step_idx=step_idx, state=state).model_dump()
    for step_idx in range(Np) for state in [state for state in MedState]
])

nodes_df = pd.DataFrame(result)
nodes_df_separated = pd.concat([nodes_sfts_df, nodes_med_df], axis=0)

display(nodes_df.head())
display(nodes_df_separated.head())

# Build transition/connection/edges dataframe

# step_idx = 0
# 
# edges_list = []
# for step_idx in range(Np):
#     edges_list = generate_edges(edges_list, step_idx, system='SolarMED', Np=Np)
#     
# # Convert to dataframe
# edges_df = generate_edges_dataframe(edges_list)
# 
# display(edges_df.head())

,step_idx,state,node_id,state_value,state_name,x_pos,y_pos
0,0,SolarMED_State.sf_IDLE_ts_IDLE_med_OFF,step000_000,000,sf_IDLE_ts_IDLE_med_OFF,0,0
1,0,SolarMED_State.sf_IDLE_ts_IDLE_med_GENERATING_...,step000_001,001,sf_IDLE_ts_IDLE_med_GENERATING_VACUUM,0,1
2,0,SolarMED_State.sf_IDLE_ts_IDLE_med_IDLE,step000_002,002,sf_IDLE_ts_IDLE_med_IDLE,0,2
3,0,SolarMED_State.sf_IDLE_ts_IDLE_med_STARTING_UP,step000_003,003,sf_IDLE_ts_IDLE_med_STARTING_UP,0,3
4,0,SolarMED_State.sf_IDLE_ts_IDLE_med_SHUTTING_DOWN,step000_004,004,sf_IDLE_ts_IDLE_med_SHUTTING_DOWN,0,4


,step_idx,state,node_id,state_value,state_name,x_pos,y_pos
0,0,SF_TS_State.IDLE,step000_00,00,IDLE,0,0.0
1,0,SF_TS_State.RECIRCULATING_TS,step000_01,01,RECIRCULATING_TS,0,1.0
2,0,SF_TS_State.HEATING_UP_SF,step000_10,10,HEATING_UP_SF,0,2.0
3,0,SF_TS_State.SF_HEATING_TS,step000_11,11,SF_HEATING_TS,0,3.0
4,1,SF_TS_State.IDLE,step001_00,00,IDLE,1,0.0


In [6]:
from enum import Enum
import copy

print(MedState)

node_colors = {
    str(MedState.__name__): "#c061cb",
    str(SF_TS_State.__name__): "#ff7800",
    str(SolarMED_State.__name__): '#6959CD'
}

print(MedState)

<enum 'MedState'>
<enum 'MedState'>


In [7]:
from ipywidgets import interact, widgets
from solarMED_optimization.path_exploration import generate_edges_coordinates
from solarMED_optimization.visualization import plot_state_graph, get_coordinates_edge

# generate_edges_coordinates(nodes_df, edges_df=None)

fig = plot_state_graph(nodes_df, edges_df=None, system='SolarMED', Np=Np, height=1200)

options_avg = round(len(all_paths) / len(SF_TS_State))

@interact(initial_state=[state for state in SolarMED_State], option_idx=(0, options_avg, 1))
def add_path_highlight(initial_state=SolarMED_State.sf_IDLE_ts_IDLE_med_OFF, option_idx=0):

    if initial_state is None:
        # Use random module to choose an integer from 0 to len(all_paths)
        path_idx = np.random.randint(0, len(all_paths))

    else:
        # Find the path that starts from the initial state
        path_idx = [idx for idx, path in enumerate(all_paths) if path[0] == initial_state]
        path_idx = path_idx[option_idx] if option_idx < len(path_idx) else path_idx[-1]
        print(f"Selected path {path_idx}: {all_paths_str[path_idx]}")

    # Somehow build the path coordinates from the list of all_paths
    path = all_paths[path_idx]

    x = []
    y = []
    for step_idx in range(0, len(path) - 1, 1):
        # If the naming scheme changes for whatever reason it will break
        src_node_id = f'step{step_idx:03d}_{path[step_idx].value}'
        dst_node_id = f'step{step_idx + 1:03d}_{path[step_idx + 1].value}'

        x_aux, y_aux = get_coordinates_edge(src_node_id, dst_node_id, nodes_df=nodes_df)

        x += x_aux
        y += y_aux

    with fig.batch_update():
        # First deactivate departing highlights
        fig.data[-2].x = None
        fig.data[-2].y = None

        # Then use arriving trace container to include the path
        fig.data[-1].x = x
        fig.data[-1].y = y

# Add a button that will highlight the paths that arrive/leave to/from a random node
# button = widgets.Button(description='Highlight random path')
# 
# button.on_click(add_path_highlight)
# 
fig

interactive(children=(Dropdown(description='initial_state', options=(<SolarMED_State.sf_IDLE_ts_IDLE_med_OFF: …

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#6959CD', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['sf_IDLE_ts_IDLE_med_OFF', 'sf_IDLE_ts_IDLE_med_GENERATING_VACUUM',
                             'sf_IDLE_ts_IDLE_med_IDLE', 'sf_IDLE_ts_IDLE_med_STARTING_UP',
                             'sf_IDLE_ts_IDLE_med_SHUTTING_DOWN', 'sf_IDLE_ts_IDLE_med_ACTIVE',
                             'sf_IDLE_ts_ACTIVE_med_OFF', 'sf_IDLE_ts_ACTIVE_med_GENERATING_VACUUM',
                             'sf_IDLE_ts_ACTIVE_med_IDLE', 'sf_IDLE_ts_ACTIVE_med_STARTING_UP',
                             'sf_IDLE_ts_ACTIVE_med_SHUTTING_DOWN', 'sf_IDLE_ts_ACTIVE_med_ACTIVE',
                             'sf_ACTIVE_ts_IDLE_med_OFF', 'sf_ACTIVE_ts_IDLE_med_GENERATING_VACUUM',
                             'sf_ACTIVE_ts_IDLE_med_IDLE

20

In [8]:
fig = plot_state_graph([nodes_sfts_df, nodes_med_df], edges_df=None, system='SolarMED', Np=Np, height=800)

shift = len([state for state in SF_TS_State])
# options_avg = round(len(all_paths_med) / len([state for state in MedState]))
options_avg = 1000

@interact(initial_state=[state for state in SolarMED_State], option_idx=(0, options_avg, 1))
def add_path_highlight(initial_state=SolarMED_State.sf_IDLE_ts_IDLE_med_OFF, option_idx=0):

    if initial_state is None:
        # Use random module to choose an integer from 0 to len(all_paths)
        path_idx = np.random.randint(0, len(all_paths))

    else:
        # Find the path that starts from the initial state
        path_idx = [idx for idx, path in enumerate(all_paths) if path[0] == initial_state]
        path_idx = path_idx[option_idx] if option_idx < len(path_idx) else path_idx[-1]
        print(f"Selected path {path_idx}: {all_paths_str[path_idx]}")

    # Somehow build the path coordinates from the list of all_paths
    path = all_paths[path_idx]

    x_sf = []; x_med = []
    y_sf = []; y_med = []
    for step_idx in range(0, len(path) - 1, 1):
        # If the naming scheme changes for whatever reason it will break
        
        # SF-TS path
        src_node_id = f'step{step_idx:03d}_{path[step_idx].value[:-1]}'
        dst_node_id = f'step{step_idx + 1:03d}_{path[step_idx + 1].value[:-1]}'

        x_aux, y_aux = get_coordinates_edge(src_node_id, dst_node_id, nodes_df=nodes_sfts_df)
        x_sf += x_aux; y_sf += y_aux
        
        # MED path
        src_node_id = f'step{step_idx:03d}_{path[step_idx].value[-1]}'
        dst_node_id = f'step{step_idx + 1:03d}_{path[step_idx + 1].value[-1]}'

        x_aux, y_aux = get_coordinates_edge(src_node_id, dst_node_id, nodes_df=nodes_med_df, y_shift=shift)
        x_med += x_aux; y_med += y_aux

    with fig.batch_update():
        # Use the departing trace container to include the SF-TS path
        fig.data[-2].x = x_sf
        fig.data[-2].y = y_sf

        # Then use arriving trace container to include the MED path
        fig.data[-1].x = x_med
        fig.data[-1].y = y_med

# Add a button that will highlight the paths that arrive/leave to/from a random node
# button = widgets.Button(description='Highlight random path')
# 
# button.on_click(add_path_highlight)
# 
fig

# add_path_highlight(initial_state=SolarMED_State.sf_IDLE_ts_IDLE_med_OFF, option_idx=0)

interactive(children=(Dropdown(description='initial_state', options=(<SolarMED_State.sf_IDLE_ts_IDLE_med_OFF: …

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#ff7800', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['IDLE', 'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS'], dtype=object),
              'type': 'scatter',
              'uid': 'cbb0f638-77ad-4419-8b4d-bc713c1b5601',
              'x': array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]),
              'y': array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])